In [1]:
# import the MAAP package to handle queries
from maap.maap import MAAP

import get_dem

# Choose bounding box

The bounding box should be a single string of four lat/lon, in the order:
[left  bottom  right top]

In [8]:

bbox = '-118.06817 34.22169 -118.05801 34.22822'  # Mt Wilson (near JPL)
# bbox = '-124.81360 32.44506 -113.75989 42.24498' # CA and NV
# bbox = '-125.53514 25.42150  -87.06960 49.22105' # Western US (CA, WA, Chicago, New Orleans)
# bbox = '-125.41615 31.03621 -101.80916 49.17324' # Western US (CA -> CO)
# bbox =   '36.22293  6.42000   47.17150 18.70244'  # Africa
# bbox =    '6.26868 36.00380   18.57179 47.28139'  # Italy


## For testing, Run in the ADE

In [3]:
!conda env update -n base --file environment.yaml

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 23.7.4
    latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



certifi-2023.11.17   | 155 KB    |                                       |   0% 
sardem-0.11.3        | 31 KB     |                                       |   0% 

ca-certificates-2023 | 151 KB    |                                       |   0% 


openssl-3.2.0        | 2.7 MB    |                                       |   0% 

ca-certificates-2023 | 151 KB    | ###########8                          |  32% 

ca-certificates-2023 | 151 KB    | ##################################### | 100% 


certifi-2023.11.17   | 155 KB    | ###8                                  |  10% 


certifi-2023.11.17   | 155 KB    | ##############################5       |  82% 
sardem-0.11.3        | 31 KB  

In [9]:
# WARNING - only use for testing! Not DPS!
import importlib; importlib.reload(get_dem)
from time import time

# Build a DEM
start = time()

get_dem.get_dem(bbox, "output")

print(f"Time to make DEM: {time()-start} seconds")


[01/25 19:24:29] [INFO dem.py] Bounds: -118.06817 34.22169 -118.05801 34.22822
[01/25 19:24:29] [INFO cop_dem.py] Creating output/dem.tif
[01/25 19:24:29] [INFO cop_dem.py] Fetching remote tiles...
[01/25 19:24:29] [INFO cop_dem.py] Running GDAL command:
[01/25 19:24:29] [INFO cop_dem.py] gdalwarp /vsicurl/https://raw.githubusercontent.com/scottstanie/sardem/master/sardem/data/cop_global.vrt output/dem.tif -of GTiff -ot Int16 -te -118.068169999999995 34.2216900000000024 -118.058009999999996 34.2282200000000003 -tr 0.000277777777777777778 0.000277777777777777778 -s_srs "epsg:4326+3855" -t_srs "epsg:4326" -wo NUM_THREADS=4 -r nearest -wm 5000 -multi


Creating output file that is 37P x 24L.
0...10...20...30...40...50...60...70...80...90...100 - done.
Time to make DEM: 8.123727798461914 seconds


## Launch jobs in DPS

In [ ]:
# Register Algorithm from YAML file
maap = MAAP()

maap.register_algorithm_from_yaml_file("/projects/GET-DEM-v0.1.0.yml").text